In [1]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import PIL
import torchvision
import numpy
import pandas
import torch 
import torch.optim as optim
import gc
from torch.optim.lr_scheduler import StepLR
import cv2
import os
import json
import numpy as np
from transformers import BertModel, BertTokenizer
import torch
import matplotlib.pyplot as plt
from transformers import AutoTokenizer, AutoModel
from transformers import T5EncoderModel
from transformers import GPT2Tokenizer, GPT2Model
from transformers import ViTImageProcessor, ViTModel
from PIL import Image
import requests
from tqdm import tqdm
import re 
import string 

In [2]:
PATH_DATASETS = "."
PATH_JSON_TRAIN = os.path.join(PATH_DATASETS, "annotations/data/subtask2b/train.json") 
PATH_JSON_VAL = os.path.join(PATH_DATASETS, "annotations/data/subtask2b/val.json") 
PATH_JSON_DEV = os.path.join(PATH_DATASETS, "annotations/data/subtask2b/dev_subtask2b_en.json") 
PATH_JSON_TEST = "./test_data/english/en_subtask2b_test_unlabeled.json"


PATH_IMG_TRAIN = "./subtask_2b/train"
PATH_IMG_VAL = "./subtask_2b/val"
PATH_IMG_DEV = "./subtask_2b/dev"
PATH_IMG_TEST = "./test_images/subtask2b/english"

PATH_SAVE_MODEL = "subtask2b_models"
PATH_SAVE_SUBMISSION = "subtask2b_submissions"

os.makedirs(PATH_SAVE_MODEL, exist_ok=True)
os.makedirs(PATH_SAVE_SUBMISSION, exist_ok=True)

BATCH_SIZE = 8

EPOCHS_FULL = 5
LR_FULL = 1e-5

EPOCHS_FC = 5
LR_FC = 3e-6

TRAIN_ALL = True

In [3]:
data = json.load(open(PATH_JSON_TRAIN,"r",encoding='utf-8'))

print(data[0])

{'id': '35807', 'text': 'DONALD TRUMP: BARACK\\nOBAMA AND JOE BIDEN\\nWILL BE IMPLICATED IN\\nRUSSIA HOAX\\nAP Photo/Pablo Martinez Monsivais', 'image': 'prop_meme_6570.png', 'label': 'propagandistic'}


In [4]:
def preprocess(text):
    return text

In [5]:
transform = torchvision.transforms.Compose([
                #torchvision.transforms.ToPILImage(),
                #torchvision.transforms.Resize((224,224),interpolation = PIL.Image.BICUBIC),
                #torchvision.transforms.ToTensor(),
                #torchvision.transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
            ])

In [6]:
tokenizer = AutoTokenizer.from_pretrained('limjiayi/bert-hateful-memes-expanded')
processor = ViTImageProcessor.from_pretrained('google/vit-base-patch16-224-in21k',do_resize = True,do_rescale = True,do_normalize = True,image_mean = [0.5,0.5,0.5],image_std = [0.5,0.5,0.5])

In [7]:
class MyDataset(Dataset):
    
    def __init__(self, paths_json_img):
        self.filenames = []
        self.texts = []
        self.labels = []
        self.images = []
        self.ids = []
        
        for path_json, path_img in paths_json_img:
            data_train = json.load(open(path_json,"r",encoding='utf-8'))

            for x in tqdm(data_train):
                currentPath = os.path.join(path_img,x['image'])
                
                #print(currentPath)
                
                self.ids.append(x['id'])

                if 'label' in x:
                    if x['label']=="non_propagandistic":
                        self.labels.append(0)
                    else:
                        self.labels.append(1)
                else:
                    self.labels.append(0)

                text = preprocess(x['text'])
                if text is None:
                    text = ""
                self.texts.append(tokenizer(text,return_tensors='pt',padding='max_length',max_length=128,truncation=True))
                self.filenames.append(x['image'])

                currentImage = cv2.imread(currentPath)
                currentImage = torch.tensor(transform(currentImage)).unsqueeze(0)
                features = processor(currentImage)
                self.images.append(features)

    def __len__(self):
        return len(self.images)
    
    def __getitem__(self,idx):
        
        text_tensors = {}
        for key, value in self.texts[idx].items():
            text_tensors[key] = value.cuda() if isinstance(value, torch.Tensor) else value
        
        
        image_tensors = {}
        for key, value in self.images[idx].items():
            image_tensors[key] = value.cuda() if isinstance(value, torch.Tensor) else value
            
        return ((image_tensors,text_tensors),self.labels[idx])

In [8]:
train_data = MyDataset([(PATH_JSON_TRAIN, PATH_IMG_TRAIN), (PATH_JSON_VAL, PATH_IMG_VAL),(PATH_JSON_DEV, PATH_IMG_DEV)])
valid_data = MyDataset([(PATH_JSON_VAL, PATH_IMG_VAL)])
test_data = MyDataset([(PATH_JSON_TEST, PATH_IMG_TEST)])

train_dataloader = DataLoader(dataset = train_data, batch_size = BATCH_SIZE, shuffle = True)
valid_dataloader = DataLoader(dataset = valid_data, batch_size = BATCH_SIZE, shuffle = False)
test_dataloader = DataLoader(dataset = test_data, batch_size = 1, shuffle = False)

100%|████████████████████████████████████████████████████████████████████████████████| 600/600 [00:14<00:00, 40.39it/s]


In [9]:
# print(len(train_data.images[0]['pixel_values']))
# print(len(train_data))
# print(train_data.texts[260]['input_ids'].shape)

1
1650
torch.Size([1, 128])


In [10]:
model_name = 'limjiayi/bert-hateful-memes-expanded'  
text_tokenizer = AutoTokenizer.from_pretrained(model_name)
text_model = AutoModel.from_pretrained(model_name)

C:\Users\ionut\anaconda3\envs\mlenvfinal\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertModel were not initialized from the model checkpoint at limjiayi/bert-hateful-memes-expanded and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
      
        self.text_encoder = AutoModel.from_pretrained('limjiayi/bert-hateful-memes-expanded')
        
        self.image_encoder = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')
        
        self.fc = nn.Linear(249600, 2) 
        self.fc2 = nn.Linear(128,2)
    def forward(self,  images,text_input):
        # Process text input
        
        text_outputs = []

        for i in range(text_input['input_ids'].shape[0]):
            x = dict()
            x['input_ids'] = text_input['input_ids'][i]
            x['token_type_ids'] = text_input['token_type_ids'][i]
            x['attention_mask'] = text_input['attention_mask'][i]
            text_outputs.append(self.text_encoder(**x).last_hidden_state)
            
            
        text_outputs = torch.stack(text_outputs)
        image_outputs = []
        
        for i in range(images['pixel_values'][0].shape[0]):
            x = dict()
            x['pixel_values'] = images['pixel_values'][0][i].unsqueeze(0).cuda()
          
            image_outputs.append(self.image_encoder(**x).last_hidden_state)
        
        image_outputs = torch.stack(image_outputs)


        text_outputs = text_outputs.view(text_outputs.size(0), -1)
        
        image_outputs = image_outputs.view(image_outputs.size(0), -1)
        combined = torch.cat((text_outputs, image_outputs), dim=1)
        
    
        output = self.fc(nn.Tanh()(combined))
        return output

In [12]:
print(np.sum(train_data.labels))
print(np.shape(train_data.labels))

1100
(1650,)


In [27]:
total_samples = len(train_data)
print(total_samples)
print(np.sum(train_data.labels))

class_sample_counts = [450, 900]  
class_weights = [total_samples / (len(class_sample_counts) * count) for count in class_sample_counts]
class_weights = torch.tensor(class_weights, dtype=torch.float).to('cuda')

1650
1100


In [28]:
model = Model()
model.cuda()
model.train()

C:\Users\ionut\anaconda3\envs\mlenvfinal\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of BertModel were not initialized from the model checkpoint at limjiayi/bert-hateful-memes-expanded and are newly initialized: ['bert.pooler.dense.weight', 'bert.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model(
  (text_encoder): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_aff

In [29]:
criterion = torch.nn.CrossEntropyLoss(weight = class_weights)

In [30]:
optimizer = torch.optim.Adam(model.parameters(), lr = LR_FULL)

In [31]:
best_loss = 1e9

for epoch in range(EPOCHS_FULL):

    train_loss = 0.0    
    model.train()
    for useless_id, ((images_batch, texts_batch), labels_batch) in tqdm(enumerate(train_dataloader)):
        optimizer.zero_grad()
        
        labels_batch = labels_batch.type(torch.LongTensor)
        labels_batch = labels_batch.to('cuda')
        
        labels_predictions = model(images_batch, texts_batch)
        
        loss = criterion(labels_predictions, labels_batch)
        loss.backward()
        
        optimizer.step()
        
        train_loss = train_loss + loss.item()
    
    # Validation loop
    validation_loss = 0.0
    model.eval()
    correct = 0
    total = 0
    
    for useless_id, ((images_batch, texts_batch), labels_batch) in tqdm(enumerate(valid_dataloader)):
        labels_batch = labels_batch.to('cuda')
        labels_predictions = model(images_batch, texts_batch)
        
        loss = criterion(labels_predictions, labels_batch)
       
        
        validation_loss = validation_loss + loss.item()
        
        
        _, predicted = torch.max(labels_predictions, 1)
        total += labels_batch.size(0)
        correct += (predicted == labels_batch).sum().item()
        

    train_loss /= len(train_dataloader.dataset)
    validation_loss /= len(train_dataloader.dataset)
    accuracy = correct / total
    print(f'Epoch: {epoch} Train Loss: {train_loss} Validation Loss: {validation_loss} Validation Accuracy: {accuracy * 100:.2f}%')
        
    # Save checkpoint if needed
    # checkpoint = {'checkpoint': model.state_dict()}
    # torch.save(checkpoint, os.path.join(PATH_SAVE_MODEL, f'checkpoint_{epoch}.pt'))
    print(f'Checkpoint reached! Validation loss modified from {best_loss} to {validation_loss}')
    best_loss = validation_loss
    torch.cuda.empty_cache()

207it [01:24,  2.45it/s]
19it [00:12,  1.51it/s]


Epoch: 0 Train Loss: 0.06211537418491912 Validation Loss: 0.0031594300044305397 Validation Accuracy: 90.00%
Checkpoint reached! Validation loss modified from 1000000000.0 to 0.0031594300044305397


207it [05:54,  1.71s/it]
19it [00:12,  1.57it/s]


Epoch: 1 Train Loss: 0.028244069501983396 Validation Loss: 0.0011206580021164633 Validation Accuracy: 99.33%
Checkpoint reached! Validation loss modified from 0.0031594300044305397 to 0.0011206580021164633


183it [07:19,  2.40s/it]


KeyboardInterrupt: 

In [ ]:
for param in model.text_encoder.parameters():
    param.requires_grad = False
    
for param in model.image_encoder.parameters():
    param.requires_grad = False 
    

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = LR_FC)

In [ ]:
best_loss = 1e9


for epoch in range(EPOCHS_FC):

    train_loss = 0.0    
    model.train()
    for useless_id, ((images_batch, texts_batch), labels_batch) in tqdm(enumerate(train_dataloader)):
        optimizer.zero_grad()
        
        labels_batch = labels_batch.type(torch.LongTensor)
        labels_batch = labels_batch.to('cuda')
        
        labels_predictions = model(images_batch, texts_batch)
        
        loss = criterion(labels_predictions, labels_batch)
        loss.backward()
        
        optimizer.step()
        
        train_loss = train_loss + loss.item()
    
    # Validation loop
    validation_loss = 0.0
    model.eval()
    correct = 0
    total = 0
    
    for useless_id, ((images_batch, texts_batch), labels_batch) in tqdm(enumerate(valid_dataloader)):
        labels_batch = labels_batch.to('cuda')
        
        labels_predictions = model(images_batch, texts_batch)
        
        loss = criterion(labels_predictions, labels_batch)
       
        
        validation_loss = validation_loss + loss.item()
        
        
        _, predicted = torch.max(labels_predictions, 1)
        total += labels_batch.size(0)
        correct += (predicted == labels_batch).sum().item()
        

    train_loss /= len(train_dataloader.dataset)
    validation_loss /= len(train_dataloader.dataset)
    accuracy = correct / total
    print(f'Epoch: {epoch} Train Loss: {train_loss} Validation Loss: {validation_loss} Validation Accuracy: {accuracy * 100:.2f}%')
        
    # Save checkpoint if needed
    # checkpoint = {'checkpoint': model.state_dict()}
    # torch.save(checkpoint, os.path.join(PATH_SAVE_MODEL, f'fc_checkpoint_{epoch}.pt'))
    print(f'Checkpoint reached! Validation loss modified from {best_loss} to {validation_loss}')
    best_loss = validation_loss
    torch.cuda.empty_cache()

In [ ]:
checkpoint = {'checkpoint': model.state_dict()}
torch.save(checkpoint, os.path.join(PATH_SAVE_MODEL, f'checkpoint.pt'))

In [ ]:
#import torch
# model.train()
# checkpoint = torch.load(os.path.join(PATH_SAVE_MODEL, f'fc_checkpoint_{4}.pt'))

# # Apply the state dictionary to the model
# model.load_state_dict(checkpoint['checkpoint'])

In [ ]:
predictions = []
ids = []

for useless_id, ((images_batch, texts_batch), labels_batch) in tqdm(enumerate(test_dataloader)):
    model.eval()
    labels_batch = labels_batch.type(torch.LongTensor)
        
    # Move data to GPU
    #images_batch = images_batch.to('cuda')
    #texts_batch = texts_batch.to(device)
    labels_batch = labels_batch.to('cuda')

    labels_predictions = model(images_batch, texts_batch)


    _, predicted = torch.max(labels_predictions, 1)

    predictions.append(predicted.item())
    ids.append(test_data.ids[useless_id])

In [ ]:
with open(os.path.join(PATH_SAVE_SUBMISSION, "submission.txt"),"w") as fout:
    print("[\n",end='',file=fout)
    
    idx = 0
    for (ID,pred) in zip(ids,predictions):
        idx += 1
        predName = None
        if pred==0:
            predName = 'non_propagandistic'
        else:
            predName = 'propagandistic'
            
        ID = "\"" + ID + "\""
        predName = "\"" + predName + "\""

        print("{\n\"id\":",end='',file=fout)
        print(f" {ID},\n",end='',file=fout)
        print("\"label\":",end='',file=fout)
        print(f" {predName}\n",end='',file=fout)
        
        
        if idx < len(predictions):
            print("},",file=fout)
        else:
            print("}",file=fout)
    print(']',file=fout)